In [1]:
import jupyros as jr
import rospy
import actionlib
import actionlib.msg
import assignment_2_2023
import assignment_2_2023.msg
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt

from std_srvs.srv import *
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
from assignment_2_2023.msg import pos_vel
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation

%matplotlib widget

In [2]:
#define parameters
my_data = pos_vel()
target_reached = 0
target_cancelled = 0 
cancelled = False 
target = None

In [3]:
def callback(data):
   
    global pub, my_data
    my_data.y = data.pose.pose.position.y
    my_data.vel_x = data.twist.twist.linear.x
    my_data.vel_z = data.twist.twist.linear.z
  
    pub.publish(my_data)

In [4]:
#laser callback
def laser_callback(msg):
    global last_laser
    initial_loc = len(msg.ranges) // 2 - (len(msg.ranges) // 4)
    end_loc = len(msg.ranges) // 2 + (len(msg.ranges) // 4)
    ranges = msg.ranges[initial_loc:end_loc]
    min_dist = min(ranges)

In [5]:
#Creating button state and style

set_target_button_style = {'button_color': 'lightgreen'}
cancel_target_button_style = {'button_color': 'lightcoral'}

set_target_button = widgets.Button(value=False, description="Set Target",
                                   disabled=False, style=set_target_button_style)
cancel_target_button = widgets.Button(value=False, description="Cancel Target",
                                      disabled=False, style=cancel_target_button_style)

In [6]:
#Convert button input
x = widgets.BoundedFloatText(value=0.0, description='x', min=-9.0, max=9.0,
                              style={'description_width': 'initial'}, step=0.1,
                              layout=widgets.Layout(width='100px'))
y = widgets.BoundedFloatText(value=0.0, description='y', min=-9.0, max=9.0,
                              style={'description_width': 'initial'}, step=0.1,
                              layout=widgets.Layout(width='100px'))


In [7]:
def callback_send(msg):
    global cancelled, target
    target = assignment_2_2023.msg.PlanningGoal()
    target.target_pose.pose.position.x = x.value
    target.target_pose.pose.position.y = y.value
    cancelled = False
    client.send_goal(target)

In [8]:
def callback_cancel(msg):
    global cancelled, target
    target = None
    cancelled = True
    client.cancel_goal()

In [9]:
set_target_button.on_click(callback_send)
cancel_target_button.on_click(callback_cancel)

## Position Visualization:

In [10]:
class Odom_Interface:
    def __init__(self):
        self.fig_odom, self.ax = plt.subplots()
        # Robot Position Plot
        self.ln, = plt.plot([], [], 'bo', label='Robot Position')  # Blue empty circle
        # Target's Position Plot
        self.goal_ln, = plt.plot([], [], 'g*', markersize=10, label='Robot Goal Position')  # Green star
        # Robot's Path Plot
        self.path_ln, = plt.plot([], [], 'b', label='Robot Path')  # Blue line
        # Robot's Position Data Arrays
        self.x_data, self.y_data = [], []
        self.path_x, self.path_y = [], []

    def plot_init(self):
        # Set Plot Title
        self.ax.set_title("Robot Odometry", fontsize=20, fontweight='bold')
        # Set Plot Axis Labels
        self.ax.set_xlabel("X [m]", fontsize=10, fontweight="bold")
        self.ax.set_ylabel("Y [m]", fontsize=10, fontweight="bold")
        # Set Plot Axis Limits
        self.ax.set_xlim(-20, 20)
        self.ax.set_ylim(-20, 20)
        # Set Grid to True
        self.ax.grid(True)
        return self.ln, self.goal_ln, self.path_ln

    def odom_callback(self, msg):
        # Callback Function used to update data
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        # Add current position to the path
        self.path_x.append(msg.pose.pose.position.x)
        self.path_y.append(msg.pose.pose.position.y)

    def update_plot(self, frame):
        # Update Robot's Position on plot
        self.ln.set_data(self.x_data[-1], self.y_data[-1])

        if cancelled:
            # Show position with a red full circle if cancelled
            self.goal_ln.set_data(self.x_data[-1], self.y_data[-1])
            self.goal_ln.set_marker('o')  # Full circle
            self.goal_ln.set_color('red')  # Red color
        elif target is not None:
            # Show the final position with a green star
            self.goal_ln.set_data(target.target_pose.pose.position.x, target.target_pose.pose.position.y)
            self.goal_ln.set_marker('*')  # Star shape
            self.goal_ln.set_color('green')  # Green color
        else:
            self.goal_ln.set_data([], [])

        # Update Robot's Path on plot
        self.path_ln.set_data(self.path_x, self.path_y)

        return self.ln, self.goal_ln, self.path_ln


## Number of Reached/Cancelled Goals Visualization:

In [11]:
class Goal_Interface:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        # Setting up the values
        self.reached = 0
        self.cancelled = 0
        # Set Plot Title
        self.ax.set_title("Reached/Cancelled Goals", fontsize=20, fontweight="bold")

        self.ax.grid(axis='y', color='grey', linestyle='-', alpha=0.5)
        # Set Plot Labels on x-axis
        self.labels = ('Reached', 'Cancelled')
        self.x_pos = np.arange(len(self.labels))
        self.ax.set_xticks(self.x_pos)
        self.ax.set_xticklabels(self.labels)
        # Set Plot Axis Limits
        self.ax.set_ylim([0, 20])
        self.ax.set_yticks(np.arange(0, 21, 1))
        self.bar_colors = ['lightgreen', 'lightcoral']
        self.bar_plot = self.ax.bar(self.x_pos, [self.reached, self.cancelled], align='center', color=self.bar_colors, width=1)

    def goal_callback(self, msg):
        # Get the number of reached/cancelled goals
        global target_reached, target_cancelled
        if msg.status.status == 3:
            self.reached += 1
        elif msg.status.status == 2:
            self.cancelled += 1

    def update_plot(self, frame):
        self.green_val = np.random.randint(0, 100)
        self.red_val = np.random.randint(0, 100)

        for i, bar in enumerate(self.bar_plot):
            if i == 0:
                bar.set_height(self.reached)
            else:
                bar.set_height(self.cancelled)
            bar.set_color(self.bar_colors[i])
        return self.bar_plot

## Initialize the Node:

In [12]:
rospy.init_node('jupyter_notebook')
pub = rospy.Publisher("/pos_vel", pos_vel, queue_size=1)
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
client.wait_for_server()

True

# Main:

## Set target position:

In [13]:
display(widgets.HBox([x, y]))
display(widgets.HBox([set_target_button, cancel_target_button]))

In [14]:
visualize_goals = Goal_Interface()
results = rospy.Subscriber('/reaching_goal/result', assignment_2_2023.msg.PlanningActionResult, visualize_goals.goal_callback)

animation_goal = FuncAnimation(visualize_goals.fig, visualize_goals.update_plot, interval=1000)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot robot position:

In [15]:
position_visualizer = Odom_Interface()
sub = rospy.Subscriber('/odom', Odometry, position_visualizer.odom_callback)

animation_pos = FuncAnimation(position_visualizer.fig_odom, position_visualizer.update_plot, init_func=position_visualizer.plot_init)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …